<a href="https://colab.research.google.com/github/yjodev/Tobigs19/blob/main/Week1_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

- Framework: 뼈대나 기반을 의미한다. 프로그래머는 뼈대 위에서 코드를 작성할 수 있다. 필수적인 코드와 알고리즘과 같은 구조를 제공한다. 프레임워크는 완성된 제품이 아니라 개발자가 이를 이용하여 완성된 제품을 만들 수 있도록 도와준다. 통제권을 사용자가 아닌 프레임워크가 쥐고 있다. 프레임워크의 예로는 스프링, 장고, 노드JS 등이 있다.
- Library: 특정 기능에 대한 도구 또는 함수들의 집합. 프로그램 기능 수행을 위해 활용 가능한 도구의 집합이다. 프로그래머가 자신이 원하는 기능을 구현하고 싶을 때 라이브러리를 가져다 사용할 수 있다. 라이브러리의 선택과 통제권은 사용자에게 있다. 예로는 Numpy, Pandas, Matplotplib 등이 있다. 

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

- Machine Learning: 

데이터 양이 적다면 딥러닝 알고리즘의 성능은 좋지 않은 반면 수작업이 포함된 머신러닝 알고리즘의 성능이 우세하다. 딥러닝 알고리즘은 GPU를 요구하여 고사양 기기를 요구하고, 머신러닝은 저사양 기기에서도 실행 가능하다. 머신러닝의 경우, 변수는 전문가가 식별한 뒤 데이터 유형별로 직접 코딩하여야 한다. 딥러닝 알고리즘은 high level feature를 학습하여 새로운 변수 추출이라는 작업을 줄여준다. 머신러닝 알고리즘으로 문제를 해결할 때에는 주로 문제를 여러 부분으로 나눈 뒤, 각각에 대한 답을 구하고 그 결과를 결합한다. 반면 딥러닝은 end-to-end 방식으로 문제를 해결한다. 딥러닝 알고리즘은 머신러닝 알고리즘에 비해 시간이 많이 소요된다. 해석이 필요한 산업에서는 의사결정나무나 선형회귀모형, 로지스틱회귀모형과 같은 머신러닝 알고리즘을 사용한다. 딥러닝 알고리즘을 실제 산업에서 사용하기에는 해석력이 좋지 않다. 

### Q3. 아래의 코드에 주석 달기.

In [ ]:
# 딥러닝 프레임워크 중 하나인 파이토치를 학습해보자

In [5]:
# 파이토치
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

# GPU 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(45)
if device == 'cuda':
    torch.cuda.manual_seed_all(45)
print(device + " is available")

cpu is available


In [6]:
# paramater
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

In [7]:
# 훈련용 MNIST dataset
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True, # 훈련 데이터를 return
    download = True,
    transform = transfroms.Compose([ #받아온 MNIST 데이터를 파이토치 텐서로 변환
        transfroms.ToTensor() 
    ])
)

# 테스트용 MNIST dataset
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False, # 테스트 데이터를 return
    download = True,
    transform = transfroms.Compose([  #받아온 MNIST 데이터를 파이토치 텐서로 변환
        transfroms.ToTensor()
    ])
)

In [8]:
# train_loader, test_loader 생성
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

# input size를 알기 위해서
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

torch.Size([1, 28, 28])

In [9]:
# CNN 모델 만들기

class ConvNet(nn.Module):
  def __init__(self): # 초기함수 각각의 layer를 설정한다.
        super(ConvNet, self).__init__() 
         # input size = 28x28
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # input channel = 1, filter = 10, kernel size = 5, zero padding = 0, stribe = 1
        # ((W-K+2P)/S)+1 공식으로 인해 ((28-5+0)/1)+1=24 -> 24x24로 변환
        # maxpooling하면 12x12
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # input channel = 1, filter = 10, kernel size = 5, zero padding = 0, stribe = 1
        # ((12-5+0)/1)+1=8 -> 8x8로 변환
        # maxpooling하면 4x4
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) # 랜덤하게 뉴런을 종료해서 학습을 방해해 학습이 학습용 데이터에 치우치는 현상을 막기 위해 사용
        self.mp = nn.MaxPool2d(2) # 오버피팅을 방지하고, 연산에 들어가는 자원을 줄이기 위해 maxpolling
        self.fc1 = nn.Linear(320,100)  # 4x4x20 vector로 flat한 것을 100개의 출력으로 변경
        self.fc2 = nn.Linear(100,10) # 100개의 출력을 10개의 출력으로 변경, 최종 output은 0~9 숫자이므로 10개


  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x)))  # convolution layer 1번에 relu를 씌우고 maxpool, 결과값은 12x12x10 # conv1 layer를 통과한 값이 x
        x = F.relu(self.mp(self.conv2(x)))  # convolution layer 2번에 relu를 씌우고 maxpool, 결과값은 4x4x20 # conv2 layer를 통과한 값이 x
        x = self.drop2D(x)                  # drop2D layer를 통과한 값이 x
        x = x.view(x.size(0), -1)           # flat
        x = self.fc1(x)                     # fc1 layer를 통과한 값이 x
        x = self.fc2(x)                     # fc2 layer를 통과한 값이 x
        return F.log_softmax(x)             # fully-connected layer에 넣고 logsoftmax 적용

In [10]:
# CNN 모델 생성 
model = ConvNet().to(device) 
# cross entropy Loss Function 사용
criterion = nn.CrossEntropyLoss().to(device)
# Adam Optimizer에 위에서 설정한 lerning rate를 paramater에 입력
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [11]:
# Training
for epoch in range(epochs): # 5번 반복
    avg_cost = 0
    for data, target in train_loader: # data는 인풋 (이미지), target은 라벨
        data = data.to(device)
        target = target.to(device)

        optimizer.zero_grad() # 모든 model의 gradient 값을 0으로 설정
        hypothesis = model(data) # 모델에 input값을 넣은 출력값이 가설 # 모델을 forward pass해 결과값 저장 

        cost = criterion(hypothesis, target) # 가설과 실제 라벨 사이의 차이 계산: 크로스 엔트로피 이용 # output과 target의 loss 계산
        cost.backward() # cost 값을 backword함  # backward 함수를 호출해 gradient 계산
        optimizer.step() # backword값에 맞춰서 optimizer step로 모델 학습  # 모델의 학습 파라미터 갱신
        avg_cost += cost / len(train_loader)  # average cost 계산 # loss 값을 변수에 누적하고 train_loader의 개수로 나눔 = 평균
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost)) # 에폭별 평균 cost 출력

[Epoch:    1]  cost = 0.329674006
[Epoch:    2]  cost = 0.11131534
[Epoch:    3]  cost = 0.0849394649
[Epoch:    4]  cost = 0.0731378645
[Epoch:    5]  cost = 0.0653285831


In [12]:
# test
model.eval() # evaluate mode로 전환 dropout 이나 batch_normalization 해제 
with torch.no_grad(): # 학습 없음 # grad 해제 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] # 출력이 분류 각각에 대한 값으로 나타나기 때문에, 가장 높은 값을 갖는 인덱스를 추출
        total += len(target)  # 전체 클래스 개수 
        correct += (preds==target).sum().item() # 예측값과 실제값이 같은지 비교
        
    print('Test Accuracy: ', 100.*correct/total, '%')
     

<ipython-input-9-edf09fe4524a>:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Test Accuracy:  98.75 %


참고: https://www.youtube.com/watch?v=wQtkdq3tmJ8, 

https://yong0810.tistory.com/17


## 첫 정규세션 들으시느라 고생 많으셨습니다.